In [1]:
import numpy as np
import pandas as pd
from fullfrlm import flow_refueling_location_model
import pickle
import networkx as nx
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))

In [2]:
df_h

,Unnamed: 0,country,city_abbr,city_full,function,coords,harbour_code,lat,lon,harbour_node
0,78676,NL,ABL,Alblasserdam,1-------,nan,NLABL,51.861514,4.657490,8866686
1,78678,NL,ALK,Alkmaar,123-----,5238N 00445E,NLALK,52.633333,4.750000,8861625
2,78680,NL,ALM,Almelo,123-----,5221N 00640E,NLALM,52.350000,6.666667,8866425
3,78682,NL,AER,Almere,1-3--6--,5222N 00513E,NLAER,52.366667,5.216667,8864005
4,78685,NL,APN,Alphen aan den Rijn,123-----,5208N 00440E,NLAPN,52.133333,4.666667,8863360
...,...,...,...,...,...,...,...,...,...,...
65,80378,NL,WTE,West-Terschelling,1-------,nan,NLWTE,53.362705,5.216893,8865605
66,80438,NL,WMO,Wormer,1-3-----,5230N 00449E,NLWMO,52.500000,4.816667,8866889
67,80450,NL,ZAA,Zaandam,123-----,nan,NLZAA,52.442040,4.829199,8866889
68,80514,NL,ZWI,Zwijndrecht,123-----,nan,NLZWI,51.810598,4.627272,8865003


In [4]:
total_flow, fraction_captured_total, serveable_fraction, served_fraction, optimal_facilities, non_zero_flows, routes_supported, paths, G, df_nodes_abm, df_links_abm = flow_refueling_location_model(1, 50000, 100, 5, 5, 20, vis=False)

Optimal


In [5]:
nodes = []
for route in non_zero_flows.keys():
    nodes += paths[route]

nodes = list(set(nodes))

In [6]:
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [7]:
df_nodes = df_nodes.loc[df_nodes.index.isin(nodes)]

In [8]:
df_nodes.fillna(0, inplace=True)
df_nodes['node_type'] =df_nodes.n.apply(lambda x: "harbour" if ((len(str(x))>4) and (x in df_h.harbour_node.unique())) else x)
df_nodes.node_type = df_nodes.node_type.apply(lambda x: "inserted_node" if x==0 else x)
df_nodes.node_type = df_nodes.node_type.apply(lambda x: "intermediate_node" if str(x).isdigit() else x)

In [9]:
df_nodes

,n,X,Y,geometry,Wkt,node_type
30986055,30986055,5.402429,53.177448,"{'type': 'Point', 'coordinates': (5.4024286549...",POINT (5.4024286549502500 53.1774479247266001),intermediate_node
8866889,8866889,4.832089,52.431155,"{'type': 'Point', 'coordinates': (4.8320885330...",POINT (4.8320885330888101 52.4311547312740984),harbour
30984595,30984595,4.572310,51.692559,"{'type': 'Point', 'coordinates': (4.5723099808...",POINT (4.5723099808310499 51.6925587903231971),harbour
8862690,8862690,5.525170,52.500915,"{'type': 'Point', 'coordinates': (5.5251701716...",POINT (5.5251701716520598 52.5009152734822990),intermediate_node
8867901,8867901,5.765900,52.583854,"{'type': 'Point', 'coordinates': (5.7659000453...",POINT (5.7659000453479203 52.5838536996871895),intermediate_node
...,...,...,...,...,...,...
8863742,8863742,4.553617,51.905834,"{'type': 'Point', 'coordinates': (4.5536169332...",POINT (4.5536169332697396 51.9058337674907975),intermediate_node
8864837,8864837,5.021452,52.278257,"{'type': 'Point', 'coordinates': (5.0214516249...",POINT (5.0214516249353398 52.2782565774994978),harbour
104,0,4.855249,51.945632,0,0,inserted_node
105,0,4.503940,52.050725,0,0,inserted_node


In [10]:
df_nodes['index1'] = df_nodes.index
df_nodes['charging_stations'] = df_nodes.index1.apply(lambda x: optimal_facilities[x] if (x in df_h.harbour_node.unique()) or (len(str(x)) == 3) else 0)
df_nodes.drop(columns=['index1', 'n', 'geometry', 'Wkt'], inplace=True)

In [11]:
df_nodes

,X,Y,node_type,charging_stations
30986055,5.402429,53.177448,intermediate_node,0.0
8866889,4.832089,52.431155,harbour,1.0
30984595,4.572310,51.692559,harbour,2.0
8862690,5.525170,52.500915,intermediate_node,0.0
8867901,5.765900,52.583854,intermediate_node,0.0
...,...,...,...,...
8863742,4.553617,51.905834,intermediate_node,0.0
8864837,5.021452,52.278257,harbour,5.0
104,4.855249,51.945632,inserted_node,2.0
105,4.503940,52.050725,inserted_node,1.0


In [12]:
df_links = nx.to_pandas_edgelist(G)

In [14]:
df_links.loc[:, ['source', 'target', 'length_m']]

,source,target,length_m
0,30986055,8867600,8380.518917
1,30986055,8864167,68.700608
2,30986055,8865756,2481.139176
3,11309058,8865990,18936.187403
4,11309058,8866775,11560.215727
...,...,...,...
229,8868457,8865470,1299.095559
230,8863207,102,33576.255098
231,8861231,8865449,366.299833
232,8861231,104,28372.871466
